- This model is to output the feasibility

In [1]:
import json
import sys
sys.path.append("../src")

from model_function import build_graph_from_sample

# Construct the GNN Data
with open("../data/casuals_feas.json") as f:
    casuals = json.load(f)

# Remove the climbing point <= 6
casuals = [c for c in casuals if c["features"]["n_points"]>6]

graph_list = [build_graph_from_sample(sample) for sample in casuals]

In [6]:
import torch.optim as optim
from model_function import ClimbGNN
import torch.nn as nn

model = ClimbGNN()
# Use Adam optimizer to update the model parameters
optimizer = optim.Adam(model.parameters(),lr=0.01)

# loss function to calculate the loss
loss_fn = nn.CrossEntropyLoss()

for epoch in range(10):
    total_loss = 0
    correct = 0
    for graph in graph_list:
        # Clear last round gradient
        optimizer.zero_grad()
        # forward: get the predicted result
        out = model(graph)
        # calculate the loss between predicted value and real label
        loss = loss_fn(out,graph.y)
        # calculate the gradient
        loss.backward()
        # Use gradient to update the model parameters
        optimizer.step()
        total_loss += loss.item()

        pred = model(graph).argmax(dim=1)
        correct += int(pred == graph.y)

    print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}, Accuracy: {correct / len(graph_list):.4f}")

Epoch 1: Loss = 1794.3580, Accuracy: 0.9649
Epoch 2: Loss = 694.3920, Accuracy: 0.9653
Epoch 3: Loss = 689.1733, Accuracy: 0.9653
Epoch 4: Loss = 687.7087, Accuracy: 0.9653
Epoch 5: Loss = 686.7751, Accuracy: 0.9653
Epoch 6: Loss = 685.5256, Accuracy: 0.9653
Epoch 7: Loss = 683.4602, Accuracy: 0.9653
Epoch 8: Loss = 682.2746, Accuracy: 0.9653
Epoch 9: Loss = 682.2736, Accuracy: 0.9653
Epoch 10: Loss = 682.2736, Accuracy: 0.9653


In [ ]:
import torch

# Save the model
torch.save(model.state_dict(),"climb_gnn_model.pth")

In [ ]:
from model_function import predict_feasibility
from model_function import ClimbGNN
import torch

# Load
model = ClimbGNN()
model.load_state_dict(torch.load("climb_gnn_model.pth"))

route = [[1360.3333333333333, 1839.9166666666667],
    [1327.3809523809523, 1560.5714285714287],
    [1023.0, 1340.9],
    [1318.4545454545455, 1128.909090909091],
    [1166.8666666666666, 778.4],
    [934.3636363636364, 722.1818181818181],
    [1225.4545454545455, 542.3636363636364],
    [974.8333333333334, 471.1666666666667],
    [918.425, 370.75],
    [512.7142857142857, 278.14285714285717]]

climber_info = {
    'height': 179.3,
    'weight': 63.9,
    'ape_index': 1.08,
    'strength': 98,
    'flexibility': 8
}

feasible = predict_feasibility(model,route,climber_info)
print(feasible)


Not feasible


C:\Users\guanj\AppData\Local\Temp\ipykernel_34544\1830769959.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("climb_gnn_model.pth"))
